In [62]:
import os
import numpy as np
import json

In [63]:
X = []
Y = []

pre_building_files = sorted(os.listdir('data/train_images/pre_buildings'))
post_building_files = sorted(os.listdir('data/train_images/post_buildings'))

In [64]:
json_source = 'data/train_labels/post_labels/'
post_hurricane_jsons = sorted([f for f in os.listdir(json_source)])

damage_labels = []
for image_json in post_hurricane_jsons[:200]:
    with open (json_source + image_json) as f:
        json_data = json.load(f)
        
    for building in json_data['features']['xy']:
        damage_labels.append(building['properties']['subtype'])


In [65]:
from PIL import Image

Images = []
for f in pre_building_files:
    Images.append(Image.open('data/train_images/pre_buildings/' + f).convert('RGB'))

In [66]:
X_pre = []
for I in Images:
    X_pre.append(np.asarray(I))

In [60]:
X_pre = np.array(X_pre)
X_pre.shape

(4179, 50, 50, 3)

In [67]:
Images = []
for f in post_building_files:
    Images.append(Image.open('data/train_images/post_buildings/' + f).convert('RGB'))

In [68]:
X_post = []
for I in Images:
    X_post.append(np.asarray(I))

In [69]:
X_post = np.array(X_post)
X_post.shape

(4179, 50, 50, 3)

In [70]:
X = np.concatenate((X_pre, X_post), axis=3)
X.shape

(4179, 50, 50, 6)

In [71]:
len(damage_labels)

4179

In [72]:
label_map = {'no-damage': 0, 'minor-damage': 1, 'major-damage': 2, 'destroyed': 3, 'un-classified': 4}

for label in damage_labels:
    Y.append(label_map[label])

In [73]:
train_X = X.astype('float32')
train_X /= 255

train_Y = Y

train_X.shape

(4179, 50, 50, 6)

In [79]:
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm

model = Sequential()

In [80]:
# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(50, 50, 6)))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))


# Defining loss and optimizer
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
batch_size = 64
EPOCHS = 5

# test the network:
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 3343 samples, validate on 836 samples
Epoch 1/5
3343/3343 [==============================] - 33s 10ms/step - loss: 0.6571 - acc: 0.2312 - val_loss: 0.7602 - val_acc: 0.3182
Epoch 2/5
3343/3343 [==============================] - 32s 10ms/step - loss: 0.6103 - acc: 0.3422 - val_loss: 1.0359 - val_acc: 0.0478
Epoch 3/5
3343/3343 [==============================] - 32s 10ms/step - loss: 0.5807 - acc: 0.3099 - val_loss: 0.9700 - val_acc: 0.1077
Epoch 4/5
3343/3343 [==============================] - 32s 9ms/step - loss: 0.5068 - acc: 0.3431 - val_loss: 0.4941 - val_acc: 0.6711
Epoch 5/5
 576/3343 [====>.........................] - ETA: 24s - loss: 0.4831 - acc: 0.4427

In [47]:
loss, acc = model.evaluate(train_X, train_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

110/110 [==============================] - 1s 12ms/step

Got 100.00% accuracy


In [49]:
train_Y.count(0)

110